## Libraries and Util

In [ ]:
import itertools
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score


def transformToList(inputString):
    s = inputString.translate(str.maketrans({'{': '[', '}': ']', '(': '[', ')': ']'}))
    s = s.replace('[', ' [ ')
    s = s.replace(']', ' ] ')
    s = s.replace(',', ' , ')
    words = s.split()
    output = ""
    for word in words:
        if word=="[" or word=="]" or word==",": output+=word
        else: output+='"'+word+'"'
    out = json.loads(output)
    try:
        a = np.array(out).astype(np.float).tolist()
    except:
        a = out
    return a

## Week 1

### Confusion Matrix and Metrics

#### Code

In [ ]:
class week1:
    def basic_metrics(y_true, y_pred, class_names, normalize=False):
        cm = confusion_matrix(y_true, y_pred)
        cm = cm[:,::-1][::-1]
        np.set_printoptions(precision=4)

        title='Confusion matrix'
        cmap=plt.cm.Blues
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, cm[i, j],
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        print(classification_report(y_true, y_pred, target_names=class_names[::-1],digits=4))
        plt.show()
        
    def setParams():
        class_names = transformToList(input("Enter names of classes in descending order eg [Two, One, Zero]:  "))
        y_true = transformToList(input("Enter true value of labels eg [1,1,0,1,0,1,1,2] 1D:  "))
        y_pred = transformToList(input("Enter predicted value of labels eg [1,0,1,1,0,1,0,2] 1D:  "))
        return class_names, y_true, y_pred

#### Run

In [ ]:
class_names, y_true, y_pred = week1.setParams()
week1.basic_metrics(y_true, y_pred, class_names)

## Week 2

### Batch Perceptron Learning

#### Code

In [ ]:
class batchPerceptronLearning:
    def fit(X, Y, a, n):
        # ------------------------------------------------------------------------------------
        # Applying Sample Normalisation:
        Norm_Y = []

        for x, y in zip(X, Y):

            # If the sample belongs to the class with label 2 or -1 (Check dataset in question to see how formatted):
            if y != 1:
                x = [i * -1 for i in x]
                x.insert(0, -1)
                Norm_Y.append(x)
            else:
                x.insert(0, 1)
                Norm_Y.append(x)

        print("Vectors used in Batch Perceptron Learning Algorithm:\n {}\n".format(Norm_Y))

        # ------------------------------------------------------------------------------------
        # Batch Perceptron Learning Algorithm:

        epoch = 1

        while True:

            updating_samples = []
            print("Epoch {}".format(epoch))

            for count, i in enumerate(range(len(Norm_Y))):

                # Knowing which value of a to use. If it is the first iteration, than use the given parameters in the 
                # question:
                a_prev = a
                print("The value of a used is {}".format(a_prev))
                y_input = Norm_Y[i]
                print("y Value used for this iteration is: {}".format(y_input))

                # Equation -> g(x) = a^{t}y
                ay = np.dot(a, y_input)
                print("The value of a^t*y for this iteration is: {}".format(ay))


                # Checking if the sample is misclassified or not:

                # If sample is misclassified:
                if ay <= 0:

                    # If this is the first sample in the epoch, add the previous value of a to the list of samples used 
                    # for the update to perform summation at the end of the epoch:
                    if count == 0:
                        print("This sample is misclassified. This sample will be used in update.\n")
                        updating_samples.append(np.array(a))
                        updating_samples.append(np.array(y_input))

                    # If sample is misclassified and IS NOT the first sample in the epoch:
                    else:
                        print("This sample is misclassified. This sample will be used in update.\n")
                        updating_samples.append(np.array(y_input))

                # If sample is classified correctly:
                else: 

                    # If first sample in the epoch, append the previous value of a to the updating samples list:
                    if count == 0:
                        updating_samples.append(np.array(a))
                        print("This sample is classified correctly.\n")
                    else:
                        print("This sample is classified correctly.\n")

            # Calculating new value of a after having gone through all of the samples in the dataset since it is Batch Learning.
            a_update_val = n * sum(updating_samples)

            # If Block to check whether learning has converged. If we have gone through all the data without needing 
            # to update the parameters, we can conclude that learning has converged.
            if len(updating_samples) <= 1:
                print("\nLearning has converged.")
                print("Required parameters of a are: {}".format(a))
                break

            # Updating a using our new value of a:
            a = a_update_val
            print("\nNew Value of a^t is: {}.\n".format(a))

            epoch += 1
        
    def setParams():
        X_train = transformToList(input("Enter features eg [[1, 5], [2, 5], [4, 1], [5, 1]] 2D:  "))
        y_train = transformToList(input("Enter labels eg [1, 1, 2, 2] 1D:  "))
        a = transformToList(input("Enter a. Usually [1, w1, w2...] eg [-25, 6, 3] 1D:  "))
        lr = float(input("Enter the learning rate eg 1.  "))
        return X_train, y_train, a, lr

#### Run

In [ ]:
X_train, y_train, a, lr = batchPerceptronLearning.setParams()
batchPerceptronLearning.fit(X_train, y_train, a, lr)

### Sequential Perceptron Learing

#### Code

In [ ]:
class sequentialPerceptronLearning:
    def fit(X, Y, a, n):
        # ------------------------------------------------------------------------------------
        # Applying Sample Normalisation:
        Norm_Y = []

        for x, y in zip(X, Y):

            # If the sample belongs to the class with label 2 or -1 (Check dataset in question to see how formatted):
            if y != 1:
                x = [i * -1 for i in x]
                x.insert(0, -1)
                Norm_Y.append(x)
            else:
                x.insert(0, 1)
                Norm_Y.append(x)

        print("Vectors used in Sequential Perceptron Learning Algorithm:\n {}\n".format(Norm_Y))


        # ------------------------------------------------------------------------------------
        # Sequential Perceptron Learning Algorithm:

        epoch = 1

        while True:

            updating_samples = []
            print("Epoch {}".format(epoch))

            # Keeping track of how many samples are correctly classified. If this variable reaches 
            # the value that is equal to the size of the dataset (len), than we know that learning 
            # has converged:
            correctly_classified_counter = 0

            # Going through all of the samples in the dataset one-by-one:
            for i in range(len(Norm_Y)):

                # This chooses which weight to use for an iteration. If first iteration, uses given starting weight 
                # as described in question:
                a_prev = a
                print("The value of a used is {}".format(a_prev))

                # Selecting sample to use:
                y_input = Norm_Y[i]
                print("y Value used for this iteration is: {}".format(y_input))

                # Equation -> g(x) = a^{t}y
                ay = np.dot(a, y_input)
                print("The value of a^t*y for this iteration is: {}".format(ay))


                # Checking if the sample is misclassified or not:

                # If sample is misclassified:
                if ay <= 0:

                    print("This sample is misclassified. This sample will be used in update.\n")
                    updating_samples.append(np.array(a))
                    updating_samples.append(np.array(y_input))

                    # Calculating new value of a using update rule for Sequential Perceptron Learning Algorithm:
                    a_update_val = n * sum(updating_samples)

                    a = a_update_val
                    print("\nNew Value of a^t is: {}.\n".format(a))

                # If the sample is correctly classified:
                else: 
                    print("This sample is classified correctly.\n")
                    correctly_classified_counter += 1
                    pass

                # Reset sample to add for update to occur:
                updating_samples = []

            # If Block to check whether learning has converged. If we have gone through all the data without needing 
            # to update the parameters, we can conclude that learning has converged.
            if correctly_classified_counter == len(Norm_Y):
                print("\nLearning has converged.")
                print("Required parameters of a are: {}.".format(a))
                break

            epoch += 1
        
    def setParams():
        X_train = transformToList(input("Enter features eg [[0, 2], [1, 2], [2, 1], [-3, 1], [-2, -1], [-3, -2]] 2D:  "))
        y_train = transformToList(input("Enter labels eg [1, 1, 1, -1, -1, -1] 1D:  "))
        a = transformToList(input("Enter a. Usually [1, w1, w2...] eg [1, 0, 0] 1D:  "))
        lr = float(input("Enter the learning rate eg 1.  "))
        return X_train, y_train, a, lr

#### Run

In [ ]:
X_train, y_train, a, lr = sequentialPerceptronLearning.setParams()
sequentialPerceptronLearning.fit(X_train, y_train, a, lr)